In [4]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from torchvision.utils import save_image
import os
from PIL import Image

sample_dir = "data/"


'''move the computations to the GPU if cuda is available, otherwise the computations will be run on CPU'''
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

'''defining model parameters'''
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100

'''download the training and test set'''
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.MNIST(root='../data', 
                                           train=True, 
                                           transform=transform,
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root='../data', 
                                          train=False, 
                                          transform=transform,
                                          download = True)

'''use dataloader to shuffle and batch the data'''
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

'''GAN contains a generator and a discriminator'''
generator = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

discriminator = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)

'''used for outputting the generated image'''
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

'''define an instance of the generator and the discriminator'''
generator = generator.to(device)
discriminator = discriminator.to(device)

'''we need to optimizer both generator and discriminator, so we define 2 optimizer.'''
g_optimizer = torch.optim.Adam(generator.parameters(), lr = 0.0001)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr = 0.0001)

'''binary cross entropy loss is used'''
criterion = nn.BCELoss() #equal to F.binary_cross_entropy(output, target)

'''training: we should train both the generator and discriminator separately'''
for epoch in range(num_epochs):
    for i, (img, label) in enumerate(train_loader):
        
        img = img.reshape(batch_size, -1).to(device)
        
        true_label = torch.ones(batch_size, 1).to(device)
        fake_label = torch.zeros(batch_size, 1).to(device)
        
        '''train discriminator'''
        '''use true images as true instances and fake images as false instances'''
        
        out = discriminator(img)
        d_loss = criterion(out, true_label)
        real_score = out
        
        latent_img = torch.randn(batch_size, latent_size).to(device)
        fake_img = generator(latent_img)
        fake_output = discriminator(fake_img)
        d_loss_fake = criterion(fake_output, fake_label)
        
        
        d_loss = d_loss + d_loss_fake
        g_optimizer.zero_grad()
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        
        '''train generator'''
        '''use ganarated instances as true instances'''
        
        '''get the loss with fake image'''
        latent_img = torch.randn(batch_size, latent_size).to(device)
        fake_img = generator(latent_img)
        fake_output = discriminator(fake_img)
        fake_score = fake_output
        
        g_loss = criterion(fake_output, true_label)
        g_optimizer.zero_grad()
        d_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        '''get mean of real_score and fake_score over batch'''
        if i%100 == 0:
            print('epoch [{}/{}], step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(img):{:.4f}, D(G(z)): {:.4f},'.
                 format(epoch, num_epochs, i, len(train_loader), d_loss.item(), g_loss.item(),
                       real_score.mean().item(), fake_score.mean().item()))
    fake_img = fake_img.reshape(fake_img.size(0), 1, 28, 28)
    img = img.reshape(img.size(0), 1, 28, 28)
    save_image(denorm(fake_img), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    img = Image.open(os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    img.show()

epoch [0/200], step [0/600], d_loss: 1.4384, g_loss: 0.7279, D(img):0.4595, D(G(z)): 0.4829,
epoch [0/200], step [100/600], d_loss: 0.7623, g_loss: 0.8732, D(img):0.8714, D(G(z)): 0.4179,
epoch [0/200], step [200/600], d_loss: 0.6157, g_loss: 1.4385, D(img):0.8359, D(G(z)): 0.2417,
epoch [0/200], step [300/600], d_loss: 0.3164, g_loss: 2.1477, D(img):0.8823, D(G(z)): 0.1191,
epoch [0/200], step [400/600], d_loss: 0.5928, g_loss: 1.8837, D(img):0.8309, D(G(z)): 0.1565,
epoch [0/200], step [500/600], d_loss: 0.1578, g_loss: 3.3119, D(img):0.9453, D(G(z)): 0.0380,
epoch [1/200], step [0/600], d_loss: 0.2873, g_loss: 2.5335, D(img):0.8850, D(G(z)): 0.0814,
epoch [1/200], step [100/600], d_loss: 0.3572, g_loss: 2.5370, D(img):0.8560, D(G(z)): 0.0851,
epoch [1/200], step [200/600], d_loss: 0.2125, g_loss: 2.8724, D(img):0.9229, D(G(z)): 0.0615,
epoch [1/200], step [300/600], d_loss: 0.5618, g_loss: 2.3044, D(img):0.8135, D(G(z)): 0.1042,
epoch [1/200], step [400/600], d_loss: 0.5452, g_loss:

epoch [14/200], step [300/600], d_loss: 0.1818, g_loss: 4.4202, D(img):0.9221, D(G(z)): 0.0183,
epoch [14/200], step [400/600], d_loss: 0.4647, g_loss: 4.8277, D(img):0.8674, D(G(z)): 0.0154,
epoch [14/200], step [500/600], d_loss: 0.0674, g_loss: 5.9805, D(img):0.9792, D(G(z)): 0.0080,
epoch [15/200], step [0/600], d_loss: 0.0666, g_loss: 5.6362, D(img):0.9810, D(G(z)): 0.0106,
epoch [15/200], step [100/600], d_loss: 0.1622, g_loss: 4.0084, D(img):0.9637, D(G(z)): 0.0411,
epoch [15/200], step [200/600], d_loss: 0.0740, g_loss: 4.3485, D(img):0.9768, D(G(z)): 0.0373,
epoch [15/200], step [300/600], d_loss: 0.2411, g_loss: 3.5542, D(img):0.9470, D(G(z)): 0.0536,
epoch [15/200], step [400/600], d_loss: 0.1289, g_loss: 3.8381, D(img):0.9639, D(G(z)): 0.0351,
epoch [15/200], step [500/600], d_loss: 0.1302, g_loss: 4.6904, D(img):0.9656, D(G(z)): 0.0249,
epoch [16/200], step [0/600], d_loss: 0.1945, g_loss: 4.7785, D(img):0.9289, D(G(z)): 0.0172,
epoch [16/200], step [100/600], d_loss: 0.12

epoch [28/200], step [500/600], d_loss: 0.2898, g_loss: 3.4906, D(img):0.9278, D(G(z)): 0.0958,
epoch [29/200], step [0/600], d_loss: 0.2867, g_loss: 5.0356, D(img):0.9437, D(G(z)): 0.0223,
epoch [29/200], step [100/600], d_loss: 0.2873, g_loss: 4.5190, D(img):0.9096, D(G(z)): 0.0423,
epoch [29/200], step [200/600], d_loss: 0.1252, g_loss: 4.7766, D(img):0.9437, D(G(z)): 0.0181,
epoch [29/200], step [300/600], d_loss: 0.3071, g_loss: 3.3915, D(img):0.9126, D(G(z)): 0.0907,
epoch [29/200], step [400/600], d_loss: 0.3251, g_loss: 3.9092, D(img):0.9085, D(G(z)): 0.0799,
epoch [29/200], step [500/600], d_loss: 0.3757, g_loss: 4.2692, D(img):0.8944, D(G(z)): 0.0361,
epoch [30/200], step [0/600], d_loss: 0.2878, g_loss: 5.1418, D(img):0.8812, D(G(z)): 0.0221,
epoch [30/200], step [100/600], d_loss: 0.2633, g_loss: 4.0217, D(img):0.9361, D(G(z)): 0.0554,
epoch [30/200], step [200/600], d_loss: 0.3072, g_loss: 4.3474, D(img):0.9208, D(G(z)): 0.0355,
epoch [30/200], step [300/600], d_loss: 0.23

epoch [43/200], step [100/600], d_loss: 0.2365, g_loss: 4.1711, D(img):0.9434, D(G(z)): 0.0584,
epoch [43/200], step [200/600], d_loss: 0.2203, g_loss: 3.4241, D(img):0.9241, D(G(z)): 0.1019,
epoch [43/200], step [300/600], d_loss: 0.3904, g_loss: 4.0106, D(img):0.8910, D(G(z)): 0.0631,
epoch [43/200], step [400/600], d_loss: 0.1946, g_loss: 4.3512, D(img):0.9385, D(G(z)): 0.0389,
epoch [43/200], step [500/600], d_loss: 0.4940, g_loss: 3.8435, D(img):0.8494, D(G(z)): 0.0856,
epoch [44/200], step [0/600], d_loss: 0.2160, g_loss: 4.8610, D(img):0.9301, D(G(z)): 0.0347,
epoch [44/200], step [100/600], d_loss: 0.1856, g_loss: 4.4008, D(img):0.9611, D(G(z)): 0.0656,
epoch [44/200], step [200/600], d_loss: 0.1463, g_loss: 4.7927, D(img):0.9468, D(G(z)): 0.0263,
epoch [44/200], step [300/600], d_loss: 0.2066, g_loss: 5.0842, D(img):0.9333, D(G(z)): 0.0213,
epoch [44/200], step [400/600], d_loss: 0.2212, g_loss: 4.4911, D(img):0.9264, D(G(z)): 0.0429,
epoch [44/200], step [500/600], d_loss: 0.

epoch [57/200], step [300/600], d_loss: 0.2713, g_loss: 4.9681, D(img):0.8849, D(G(z)): 0.0331,
epoch [57/200], step [400/600], d_loss: 0.2908, g_loss: 4.1067, D(img):0.9197, D(G(z)): 0.0581,
epoch [57/200], step [500/600], d_loss: 0.4250, g_loss: 3.4020, D(img):0.9008, D(G(z)): 0.1019,
epoch [58/200], step [0/600], d_loss: 0.3908, g_loss: 3.6932, D(img):0.9052, D(G(z)): 0.0960,
epoch [58/200], step [100/600], d_loss: 0.3262, g_loss: 3.6399, D(img):0.8889, D(G(z)): 0.0915,
epoch [58/200], step [200/600], d_loss: 0.5026, g_loss: 3.7441, D(img):0.8114, D(G(z)): 0.0833,
epoch [58/200], step [300/600], d_loss: 0.1953, g_loss: 4.4432, D(img):0.9258, D(G(z)): 0.0538,
epoch [58/200], step [400/600], d_loss: 0.3768, g_loss: 3.6067, D(img):0.9189, D(G(z)): 0.0874,
epoch [58/200], step [500/600], d_loss: 0.3688, g_loss: 4.4586, D(img):0.8609, D(G(z)): 0.0466,
epoch [59/200], step [0/600], d_loss: 0.4869, g_loss: 3.4899, D(img):0.8122, D(G(z)): 0.0955,
epoch [59/200], step [100/600], d_loss: 0.32

...

epoch [143/200], step [0/600], d_loss: 0.6660, g_loss: 1.9357, D(img):0.7790, D(G(z)): 0.2128,
epoch [143/200], step [100/600], d_loss: 0.8359, g_loss: 1.9436, D(img):0.7089, D(G(z)): 0.2377,
epoch [143/200], step [200/600], d_loss: 0.6673, g_loss: 1.8535, D(img):0.7871, D(G(z)): 0.2425,
epoch [143/200], step [300/600], d_loss: 0.8259, g_loss: 1.8078, D(img):0.6914, D(G(z)): 0.2478,
epoch [143/200], step [400/600], d_loss: 0.8263, g_loss: 1.6277, D(img):0.7433, D(G(z)): 0.2885,
epoch [143/200], step [500/600], d_loss: 0.7343, g_loss: 2.0512, D(img):0.7855, D(G(z)): 0.2022,
epoch [144/200], step [0/600], d_loss: 0.6983, g_loss: 2.1129, D(img):0.7716, D(G(z)): 0.2057,
epoch [144/200], step [100/600], d_loss: 0.6844, g_loss: 2.2225, D(img):0.7829, D(G(z)): 0.2060,
epoch [144/200], step [200/600], d_loss: 0.8888, g_loss: 1.6081, D(img):0.7322, D(G(z)): 0.3004,
epoch [144/200], step [300/600], d_loss: 0.7587, g_loss: 1.8023, D(img):0.7476, D(G(z)): 0.2672,
epoch [144/200], step [400/600], d

epoch [157/200], step [100/600], d_loss: 0.8760, g_loss: 2.0068, D(img):0.6894, D(G(z)): 0.2053,
epoch [157/200], step [200/600], d_loss: 0.7118, g_loss: 2.0892, D(img):0.7213, D(G(z)): 0.2002,
epoch [157/200], step [300/600], d_loss: 0.7797, g_loss: 1.7358, D(img):0.6982, D(G(z)): 0.2605,
epoch [157/200], step [400/600], d_loss: 0.8105, g_loss: 1.8867, D(img):0.6719, D(G(z)): 0.2503,
epoch [157/200], step [500/600], d_loss: 0.6876, g_loss: 1.3279, D(img):0.8204, D(G(z)): 0.3544,
epoch [158/200], step [0/600], d_loss: 0.7971, g_loss: 1.8926, D(img):0.7460, D(G(z)): 0.2270,
epoch [158/200], step [100/600], d_loss: 0.7679, g_loss: 1.7075, D(img):0.7405, D(G(z)): 0.2494,
epoch [158/200], step [200/600], d_loss: 0.9265, g_loss: 2.1022, D(img):0.6514, D(G(z)): 0.2026,
epoch [158/200], step [300/600], d_loss: 0.9366, g_loss: 1.7784, D(img):0.6840, D(G(z)): 0.2426,
epoch [158/200], step [400/600], d_loss: 0.8275, g_loss: 2.0102, D(img):0.6909, D(G(z)): 0.2091,
epoch [158/200], step [500/600],

epoch [171/200], step [200/600], d_loss: 0.7554, g_loss: 1.9657, D(img):0.7684, D(G(z)): 0.2174,
epoch [171/200], step [300/600], d_loss: 0.7821, g_loss: 1.7032, D(img):0.7333, D(G(z)): 0.3080,
epoch [171/200], step [400/600], d_loss: 0.8974, g_loss: 1.8009, D(img):0.7550, D(G(z)): 0.2299,
epoch [171/200], step [500/600], d_loss: 0.6772, g_loss: 1.9955, D(img):0.7550, D(G(z)): 0.2094,
epoch [172/200], step [0/600], d_loss: 1.0482, g_loss: 1.8389, D(img):0.7455, D(G(z)): 0.2614,
epoch [172/200], step [100/600], d_loss: 0.6917, g_loss: 2.0788, D(img):0.7650, D(G(z)): 0.1970,
epoch [172/200], step [200/600], d_loss: 0.7005, g_loss: 1.7993, D(img):0.7463, D(G(z)): 0.2519,
epoch [172/200], step [300/600], d_loss: 0.8913, g_loss: 1.7991, D(img):0.7079, D(G(z)): 0.2586,
epoch [172/200], step [400/600], d_loss: 0.7519, g_loss: 1.5536, D(img):0.8254, D(G(z)): 0.3010,
epoch [172/200], step [500/600], d_loss: 0.8919, g_loss: 1.6235, D(img):0.7073, D(G(z)): 0.2955,
epoch [173/200], step [0/600], d

epoch [185/200], step [300/600], d_loss: 1.0011, g_loss: 1.4977, D(img):0.7018, D(G(z)): 0.3237,
epoch [185/200], step [400/600], d_loss: 0.7245, g_loss: 1.6515, D(img):0.7449, D(G(z)): 0.2786,
epoch [185/200], step [500/600], d_loss: 0.8018, g_loss: 1.9680, D(img):0.7145, D(G(z)): 0.2212,
epoch [186/200], step [0/600], d_loss: 0.8235, g_loss: 1.9186, D(img):0.7138, D(G(z)): 0.2227,
epoch [186/200], step [100/600], d_loss: 0.8408, g_loss: 1.7547, D(img):0.7195, D(G(z)): 0.2710,
epoch [186/200], step [200/600], d_loss: 0.8183, g_loss: 1.7291, D(img):0.7132, D(G(z)): 0.2576,
epoch [186/200], step [300/600], d_loss: 0.8616, g_loss: 2.2925, D(img):0.6639, D(G(z)): 0.1731,
epoch [186/200], step [400/600], d_loss: 0.9353, g_loss: 1.6758, D(img):0.7414, D(G(z)): 0.2885,
epoch [186/200], step [500/600], d_loss: 0.9156, g_loss: 1.5568, D(img):0.6992, D(G(z)): 0.3089,
epoch [187/200], step [0/600], d_loss: 0.8580, g_loss: 1.7502, D(img):0.7416, D(G(z)): 0.2833,
epoch [187/200], step [100/600], d

epoch [199/200], step [400/600], d_loss: 0.8266, g_loss: 1.5146, D(img):0.7683, D(G(z)): 0.3057,
epoch [199/200], step [500/600], d_loss: 0.7848, g_loss: 1.5756, D(img):0.7303, D(G(z)): 0.2886,
